# Init

In [1]:
import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
from scipy import interpolate
from scipy import signal
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
import parula as par

In [5]:
import toolsdiadem.tools as dia

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
root_dir = "/home/pleroy/DATA/DIADEM/bfu"
# 011 ===== 3l +45
dir_011 = "/20181015"
nam_011 = "011_3l_05l_+45_amp_pha_32GHz.data"
dat_011 = root_dir + dir_011 + "/" + nam_011
# 012 ===== 3l -45
dir_012 = "/20181015"
nam_012 = "012_3l_05l_-45_amp_pha_32GHz.data"
dat_012 = root_dir + dir_012 + "/" + nam_012
# 013_0 ===== 7l +45
dir_013_0 = "/20181016/013_seq"
nam_013_0 = "SEQ_0_7l_+45_amp_pha_32GHz.data"
dat_013_0 = root_dir + dir_013_0 + "/" + nam_013_0
# 013_1 ===== 7l -45
dir_013_1 = "/20181016/013_seq"
nam_013_1 = "SEQ_1_7l_-45_amp_pha_32GHz.data"
dat_013_1 = root_dir + dir_013_1 + "/" + nam_013_1
# 014_0 ===== 5l +45
dir_014_0 = "/20181017/014_5l_05l"
nam_014_0 = "SEQ_0_5l_+45_amp_pha_32GHz"
dat_014_0 = root_dir + dir_014_0 + "/" + nam_014_0
# 014_1 ===== 5l +-5
dir_014_1 = "/20181017/014_5l_05l"
nam_014_1 = "SEQ_1_5l_-45_amp_pha_32GHz"
dat_014_1 = root_dir + dir_014_1 + "/" + nam_014_1

In [8]:
import os

In [9]:
x0y0 = (211, 244.5)

In [10]:
pi = np.pi
f0 = 32e9
c = 3e8
lambda0 = c / f0
k0 = 2 * np.pi / lambda0
w0 = 2 * np.pi * f0
mu = 4 * np.pi * 1e-7
epsilon = 8.854187817e-12 # 1 / (mu * c**2)

# Read data

In [11]:
ex_3l = np.genfromtxt( dat_012, skip_header=True )
ey_3l = np.genfromtxt( dat_011, skip_header=True )

In [12]:
dx = 4.69e-3
dy = 4.69e-3
fsx = 1 / dx
fsy = 1 / dy
ksx = 2 * np.pi * fsx
ksy = 2 * np.pi * fsy
Nx = 81
Ny = 81

In [13]:
xx = (ey_3l[:,0].reshape((81,81)) - x0y0[0]) / 1000
yy = (ey_3l[:,1].reshape((81,81)) - x0y0[1]) / 1000
XX = xx / lambda0
YY = yy / lambda0
Ax = np.rot90(ex_3l[:,2].reshape((81,81)), k=-1)
Ay = ey_3l[:,2].reshape((81,81))

In [14]:
myVmax = 10
myVmin = -30

In [15]:
fig, (ax0, ax1) = plt.subplots(1, 2)

im0 = ax0.pcolor(XX, YY, Ax, vmin=myVmin, vmax=myVmax, cmap=par.parula_map)
ax0.set_title("Ax")
dia.addColorBar(im0, ax0)

im1 = ax1.pcolor(XX, YY, Ay, vmin=myVmin, vmax=myVmax, cmap=par.parula_map)
ax1.set_title("Ay")
dia.addColorBar(im1, ax1)

title = nam_011 +"\n" + nam_012
fig.suptitle(title)

Text(0.5,0.98,'011_3l_05l_+45_amp_pha_32GHz.data\n012_3l_05l_-45_amp_pha_32GHz.data')

## Compute complex values

In [16]:
c_ex_3l = np.rot90(dia.getComplex(ex_3l[:,2],ex_3l[:,3]).reshape((81,81)), k=-1) + 1e-3
c_ey_3l =          dia.getComplex(ey_3l[:,2],ey_3l[:,3]).reshape((81,81))

## Add points

In [17]:
pointsToAdd = 80
c_ex_3l_extended = np.zeros((c_ex_3l.shape[0]+2*pointsToAdd, c_ex_3l.shape[1]+2*pointsToAdd), dtype=complex)
c_ex_3l_extended[pointsToAdd:pointsToAdd+Ny, pointsToAdd:pointsToAdd+Nx] = c_ex_3l[:,:]
c_ey_3l_extended = np.zeros((c_ey_3l.shape[0]+2*pointsToAdd, c_ey_3l.shape[1]+2*pointsToAdd), dtype=complex)
c_ey_3l_extended[pointsToAdd:pointsToAdd+Ny, pointsToAdd:pointsToAdd+Nx] = c_ey_3l[:,:]
hiresNx = Nx + 2 * pointsToAdd
hiresNy = Ny + 2 * pointsToAdd

In [18]:
fig, (ax0, ax1) = plt.subplots(1,2)

ax0.pcolor(20*np.log10(np.abs(np.real(c_ex_3l_extended))))
ax0.set_aspect("equal")
ax0.set_title("Re c_ex_3l_extended")

ax1.pcolor(20*np.log10(np.abs(np.imag(c_ex_3l_extended))))
ax1.set_aspect("equal")
ax1.set_title("Im c_ex_3l_extended")

/opt/Py3-6-Venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/Py3-6-Venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log10
  import sys


Text(0.5,1,'Im c_ex_3l_extended')

## Plane wave spectrum

In [19]:
hires = 0

### Define $k_x$ and $k_y$

In [20]:
if hires:
    kx = np.fft.fftshift(np.fft.fftfreq(hiresNx,dx)) * 2 * np.pi
    ky = np.fft.fftshift(np.fft.fftfreq(hiresNy,dy)) * 2 * np.pi
else:
    kx = np.fft.fftshift(np.fft.fftfreq(Nx,dx)) * 2 * np.pi
    ky = np.fft.fftshift(np.fft.fftfreq(Ny,dy)) * 2 * np.pi
    
mesh_kx, mesh_ky = np.meshgrid(kx, ky)

In [21]:
validIdx = np.where( k0**2 >= (mesh_kx**2 + mesh_ky**2) )
nonValidIdx = np.where( k0**2 < (mesh_kx**2 + mesh_ky**2) )
kz = np.zeros(mesh_kx.shape)
kz[validIdx] = np.power( k0**2 - mesh_kx[validIdx]**2 - mesh_ky[validIdx]**2, 0.5 )

### Compute $\theta$ and $\phi$

In [22]:
theta, phi = dia.getThetaPhi(mesh_kx, mesh_ky, kz, k0)

In [23]:
u = np.sin(theta) * np.cos(phi)
v = np.sin(theta) * np.sin(phi)
w = np.cos(theta)

In [24]:
if hires:
    centerU = int(Ny/2) + 1 + pointsToAdd
    centerV = int(Nx/2) + 1 + pointsToAdd
else:
    centerU = int(Ny/2) + 1
    centerV = int(Nx/2) + 1

### Compute $F_x(k_x,k_y)$ and $F_y(k_x,k_y)$

In [25]:
if hires:
    Fx = np.fft.fftshift(np.fft.fft2(c_ex_3l_extended))
    Fy = np.fft.fftshift(np.fft.fft2(c_ey_3l_extended))
else:
    Fx = np.fft.fftshift(np.fft.fft2(c_ex_3l))
    Fy = np.fft.fftshift(np.fft.fft2(c_ey_3l))

In [26]:
np.amax(np.abs(c_ey_3l))

2.6025555018262474

In [27]:
fig, (ax0, ax1) = plt.subplots(1, 2)

vmin=-30
vmax=50

im0 = ax0.pcolor(kx, ky, 20*np.log10(np.abs(Fx)), vmin=vmin, vmax=vmax)
ax0.set_title("Fx")
dia.addColorBar(im0, ax0)

im1 = ax1.pcolor(kx, ky, 20*np.log10(np.abs(Fy)), vmin=vmin, vmax=vmax)
ax1.set_title("Fy")
dia.addColorBar(im1, ax1)

title = nam_011 +"\n" + nam_012
fig.suptitle(title)

Text(0.5,0.98,'011_3l_05l_+45_amp_pha_32GHz.data\n012_3l_05l_-45_amp_pha_32GHz.data')

## Compute $E_{\theta}$ and $E_{\phi}$

Balanis, p.976

$ E(r,\theta,\phi) = j\frac{ke^{-jkr}}{2\pi r}[cos\theta\,F(k_x,k_y)] $

$ E_{\theta} = j \frac{k e^{-jkr}}{2 \pi r} (F_x\,cos\phi + F_y\,sin\phi) $

$ E_{\phi}   = j \frac{k e^{-jkr}}{2 \pi r} cos \theta (-F_x\,sin\phi + F_y\,cos\phi) $

In [63]:
E_theta    =                   Fx * np.cos(phi) + Fy * np.sin(phi)
E_phi      = np.cos(theta) * (-Fx * np.sin(phi) + Fy * np.cos(phi))
E_theta_dB = 20*np.log10(np.abs(E_theta))
E_phi_dB   = 20*np.log10(np.abs(E_phi))

E_co       = E_theta * np.cos(phi) - E_phi * np.sin(phi)
E_cross    = E_theta * np.sin(phi) + E_phi * np.cos(phi)
E_co_dB    = 20*np.log10(np.abs(E_co))
E_cross_dB = 20*np.log10(np.abs(E_cross))

Ex_ff = E_theta * np.cos(theta) * np.cos(phi) - E_phi * np.sin(phi)
Ey_ff = E_theta * np.cos(theta) * np.sin(phi) + E_phi * np.cos(phi)
Ex_ff_dB = 20*np.log10(np.abs(Ex_ff))
Ey_ff_dB = 20*np.log10(np.abs(Ey_ff))
max_ff_dB = np.amax((Ex_ff_dB, Ey_ff_dB))
min_ff_dB = max_ff_dB - 40

In [30]:
fig, (ax0, ax1) = plt.subplots(1, 2)

im0 = ax0.pcolor(XX, YY, Ex_ff_dB, vmin=min_ff_dB, vmax=max_ff_dB, cmap=par.parula_map)
ax0.set_title("Ex_ff")
dia.addColorBar(im0, ax0)

im1 = ax1.pcolor(XX, YY, Ey_ff_dB, vmin=min_ff_dB, vmax=max_ff_dB, cmap=par.parula_map)
ax1.set_title("Ey_ff")
dia.addColorBar(im1, ax1)

title = nam_011 +"\n" + nam_012
fig.suptitle(title)

Text(0.5,0.98,'011_3l_05l_+45_amp_pha_32GHz.data\n012_3l_05l_-45_amp_pha_32GHz.data')

In [31]:
fig, (ax0, ax1) = plt.subplots(1, 2)

im0 = ax0.pcolor(u, v, 20*np.log10(np.abs(E_theta)), vmax=vmax, vmin=vmin)
ax0.set_title("E_theta")
ax0.set_xlabel("u")
ax0.set_ylabel("v")
dia.addColorBar(im0, ax0)

im1 = ax1.pcolor(u, v, 20*np.log10(np.abs(E_phi)), vmax=vmax, vmin=vmin)
ax1.set_title("E_phi")
ax1.set_xlabel("u")
ax1.set_ylabel("v")
dia.addColorBar(im1, ax1)

title = nam_011 +"\n" + nam_012
fig.suptitle(title)

Text(0.5,0.98,'011_3l_05l_+45_amp_pha_32GHz.data\n012_3l_05l_-45_amp_pha_32GHz.data')

# Probe compensation I

Yaghjian *Approximate formulas for the Far Field and Gain of Open-Ended Rectangular Waveguide*

$\overline{E} = \frac{e^{jkr}}{kr} [ E_E(\theta) \, sin\phi \, \hat{e}_{\theta} 
+ E_H(\theta) \, cos\phi \, \hat{e}_{\phi}]$

In [54]:
AE = 1
a_WR28 = 7.112e-3 # WR28
b_WR28 = 3.556e-3 # WR28
a_X = 2.286e-2
b_X = 1.016e-2
a = a_WR28
b = b_WR28
k = k0

# Far field radiated by the probe when it is oriented along Y
E_V_theta, E_V_phi = dia.E_theta_phi_yaghjian(theta, phi, k, a, b, AE)
E_V_probe = np.abs((E_V_theta**2 + E_V_phi**2)**0.5)

# Far field radiated by the probe when it is oriented along X (i.e. rotated along z by pi/2, /!\ z of the probe /!\)
E_H_theta, E_H_phi = dia.E_theta_phi_yaghjian(theta, (pi/2+phi), k, a, b, AE) # rotate the probe by pi/2 around z
E_H_probe = np.abs((E_H_theta**2 + E_H_phi**2)**0.5)

In [55]:
fig, (ax0, ax1) = plt.subplots(1,2)

im = ax0.pcolor(u, v, E_V_probe, label="E_V_probe")
dia.addColorBar(im, ax0)
ax0.set_aspect('equal')
ax0.legend()
ax0.grid()
ax0.set_xlabel('u')
ax0.set_ylabel('v')

im = ax1.pcolor(u, v, E_H_probe, label="E_H_probe")
dia.addColorBar(im, ax1)
ax1.set_aspect('equal')
ax1.legend()
ax1.grid()
ax1.set_xlabel('u')
ax1.set_ylabel('v')

Text(0,0.5,'v')

**With $\theta$ and $\phi$ the polar spherical angles as described in Stuart/McCormic/Parini p.157, one gets**

$ E_{\theta}^H(\theta, -\phi) \, E_{\theta}(\theta, \phi) - E_{\phi}^H(\theta,-\phi) \, E_{\phi}(\theta, \phi)
= cos\theta \, I_H(\theta,\phi)$

$ E_{\theta}^V(\theta, -\phi) \, E_{\theta}(\theta, \phi) - E_{\phi}^V(\theta,-\phi) \, E_{\phi}(\theta, \phi)
= cos\theta \, I_V(\theta,\phi)$

i.e.

$ E_{\theta}^H(\theta, \phi) \, E_{\theta}(\theta, \phi) + E_{\phi}^H(\theta,\phi) \, E_{\phi}(\theta, \phi)
= cos\theta \, I_H(\theta,\phi)$

$ -E_{\theta}^V(\theta, \phi) \, E_{\theta}(\theta, \phi) - E_{\phi}^V(\theta,\phi) \, E_{\phi}(\theta, \phi)
= cos\theta \, I_V(\theta,\phi)$

with

$ I_V(k_x, k_y) = exp(jk_z z_0) \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} 
                    v_Y(x, y, z_0) \, exp(jk_x x + jk_y y) \, dx \, dy $

$ I_H(k_x, k_y) = exp(jk_z z_0) \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} 
                    v_X(x, y, z_0) \, exp(jk_x x + jk_y y) \, dx \, dy $
                    
If the phase of the pattern is not of interest, $z_0$ can be taken as zero.         
                    
$ E_{\theta}(\theta, \phi)=\frac{cos\theta}{\Delta (\theta,\phi)} 
[ -I_H(\theta,\phi) \, E_{\phi}^V (\theta,\phi) + I_V(\theta, \phi) \, E_{\phi}^H (\theta,\phi) ] $

$ E_{\phi}(\theta, \phi)=\frac{cos\theta}{\Delta (\theta,\phi)} 
[ -E_{\theta}^H (\theta,\phi) \, I_V(\theta,\phi) + E_{\theta}^V (\theta,\phi) \, I_H(\theta,\phi) ] $

with
                    
$ 
\begin{aligned} 
\Delta(\theta,\phi) & = E_{\theta}^H(\theta,-\phi) \, E_{\phi}^V(\theta,-\phi) -E_{\theta}^V(\theta,-\phi) \, E_{\phi}^H(\theta,-\phi)\\ 
                    & = E_{\theta}^H(\theta,\phi) \, E_{\phi}^V(\theta,\phi) -E_{\theta}^V(\theta,\phi) \, E_{\phi}^H(\theta,\phi)\\ 
\end{aligned}
$

In [56]:
Delta_theta_phi = E_H_theta * E_phi_V - E_V_theta * E_phi_H

In [36]:
fig, (ax0, ax1) = plt.subplots(1,2)

im = ax0.pcolor(u, v, np.abs(Delta_theta_phi), label="Delta_theta_phi")
dia.addColorBar(im, ax0)
ax0.set_aspect('equal')
ax0.legend()
ax0.grid()
ax0.set_xlabel('u')
ax0.set_ylabel('v')

ax1.plot(v[:,centerU], Delta_theta_phi[:,centerU],label=f"v, column {centerU}")
ax1.plot(u[centerV,:], Delta_theta_phi[centerV,:], label=f"u, row {centerV}")
ax1.legend()
ax1.grid()

## $E_{\theta}\,E_{\phi}$

In [57]:
E_theta_comp = np.cos(theta) / Delta_theta_phi * (-Fx * E_V_phi   + Fy * E_H_phi)
E_phi_comp   = np.cos(theta) / Delta_theta_phi * (-E_H_theta * Fy + E_V_theta * Fx)

E_theta_comp_dB = 20*np.log10(np.abs(E_theta_comp))
E_phi_comp_dB = 20*np.log10(np.abs(E_phi_comp))

In [38]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

ax0.plot(v[:,centerU], E_theta_dB[:,centerU], label=f"E_theta u = 0")
ax0.plot(v[:,centerU], E_theta_comp_dB[:,centerU], label=f"E_theta u = 0 comp")
ax0.legend()
ax0.grid()
#ax0.set_ylim(-60,0)
ax0.set_xlabel("v")

ax1.plot(v[:,centerU], E_phi_dB[:,centerU], label=f"E_phi u = 0")
ax1.plot(v[:,centerU], E_phi_comp_dB[:,centerU], label=f"E_phi u = 0 comp")
ax1.legend()
ax1.grid()
#ax1.set_ylim(-60,0)
ax1.set_xlabel("v")

ax2.plot(u[centerV,:], E_theta_dB[centerV,:], label=f"E_theta v = 0")
ax2.plot(u[centerV,:], E_theta_comp_dB[centerV,:], label=f"E_theta v = 0 comp")
ax2.legend()
ax2.grid()
#ax2.set_ylim(-60,0)
ax2.set_xlabel("u")

ax3.plot(u[centerV,:], E_phi_dB[centerV,:], label=f"E_phi v = 0")
ax3.plot(u[centerV,:], E_phi_comp_dB[centerV,:], label=f"E_phi v = 0 comp")
ax3.legend()
ax3.grid()
#ax3.set_ylim(-60,0)
ax3.set_xlabel("u")

Text(0.5,0,'u')

In [39]:
fig, (ax0, ax1) = plt.subplots(1, 2)

im0 = ax0.pcolor(u, v, 20*np.log10(np.abs(E_theta_comp)), vmax=vmax, vmin=vmin, cmap=par.parula_map)
ax0.set_title("E_theta_comp")
ax0.set_xlabel("u")
ax0.set_ylabel("v")
dia.addColorBar(im0, ax0)

im1 = ax1.pcolor(u, v, 20*np.log10(np.abs(E_phi_comp)), vmax=vmax, vmin=vmin, cmap=par.parula_map)
ax1.set_title("E_phi_comp")
ax1.set_xlabel("u")
ax1.set_ylabel("v")
dia.addColorBar(im1, ax1)

title = nam_011 +"\n" + nam_012
fig.suptitle(title)

Text(0.5,0.98,'011_3l_05l_+45_amp_pha_32GHz.data\n012_3l_05l_-45_amp_pha_32GHz.data')

In [40]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

ax0.plot(phi[:,centerU]*180/pi, 20*np.log10(np.abs(E_theta[:,centerU])), label=f"E theta theta = 0")
ax0.plot(phi[:,centerU]*180/pi, 20*np.log10(np.abs(E_theta_comp[:,centerU])), label=f"E theta theta = 0 comp")
ax0.legend()
ax0.grid()
#ax0.set_ylim(-60,0)
ax0.set_xlabel("phi")

ax1.plot(phi[:,centerU]*180/pi, 20*np.log10(np.abs(E_phi[:,centerU])), label=f"E phi theta = 0")
ax1.plot(phi[:,centerU]*180/pi, 20*np.log10(np.abs(E_phi_comp[:,centerU])), label=f"E phi theta = 0 comp")
ax1.legend()
ax1.grid()
#ax1.set_ylim(-60,0)
ax1.set_xlabel("phi")

ax2.plot(theta[centerV,:]*180/pi, 20*np.log10(np.abs(E_theta[centerV,:])), label=f"E theta phi = 0")
ax2.plot(theta[centerV,:]*180/pi, 20*np.log10(np.abs(E_theta_comp[centerV,:])), label=f"E theta phi = 0 comp")
ax2.legend()
ax2.grid()
#ax2.set_ylim(-60,0)
ax2.set_xlabel("theta")

ax3.plot(theta[centerV,:]*180/pi, 20*np.log10(np.abs(E_phi[centerV,:])), label=f"E phi phi = 0")
ax3.plot(theta[centerV,:]*180/pi, 20*np.log10(np.abs(E_phi_comp[centerV,:])), label=f"E phi phi = 0 comp")
ax3.legend()
ax3.grid()
#ax3.set_ylim(-60,0)
ax3.set_xlabel("theta")

Text(0.5,0,'theta')

## $E_x\,E_y$

In [46]:
Eco    = E_theta * np.cos(phi) - E_phi * np.sin(phi)
Ecross = E_theta * np.sin(phi) + E_phi * np.cos(phi)
Eco_dB    = 20 * np.log10( np.abs(Eco) )
Ecross_dB = 20 * np.log10( np.abs(Ecross) )

Eco_comp    = E_theta_comp * np.cos(phi) - E_phi_comp * np.sin(phi)
Ecross_comp = E_theta_comp * np.sin(phi) + E_phi_comp * np.cos(phi)
Eco_comp_dB    = 20 * np.log10( np.abs(Eco_comp) )
Ecross_comp_dB = 20 * np.log10( np.abs(Ecross_comp) )


max_dB = np.amax((Eco_comp_dB, Ecross_comp_dB))
min_dB = max_dB - 40

In [47]:
fig, ((ax0, ax1),(ax2,ax3)) = plt.subplots(2, 2)

im0 = ax0.pcolor(XX, YY, Eco_dB, vmin=min_dB, vmax=max_dB, cmap=par.parula_map)
ax0.set_title("Eco")
dia.addColorBar(im0, ax0)

im1 = ax1.pcolor(XX, YY, Ecross_dB, vmin=min_dB, vmax=max_dB, cmap=par.parula_map)
ax1.set_title("Ecross")
dia.addColorBar(im1, ax1)

im2 = ax2.pcolor(XX, YY, Eco_comp_dB, vmin=min_dB, vmax=max_dB, cmap=par.parula_map)
ax2.set_title("Eco_comp")
dia.addColorBar(im2, ax2)

im3 = ax3.pcolor(XX, YY, Ecross_comp_dB, vmin=min_dB, vmax=max_dB, cmap=par.parula_map)
ax3.set_title("Ecross_comp")
dia.addColorBar(im3, ax3)

title = nam_011 +"\n" + nam_012
fig.suptitle(title)

Text(0.5,0.98,'011_3l_05l_+45_amp_pha_32GHz.data\n012_3l_05l_-45_amp_pha_32GHz.data')

In [48]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

ax0.plot(v[:,centerU], Eco_dB[:,centerU], label=f"Eco u = 0")
ax0.plot(v[:,centerU], Eco_comp_dB[:,centerU], label=f"Eco_comp u = 0 comp")
ax0.legend()
ax0.grid()
#ax0.set_ylim(-60,0)
ax0.set_xlabel("v")

ax1.plot(v[:,centerU], Ecross_dB[:,centerU], label=f"Ecross_ u = 0")
ax1.plot(v[:,centerU], Ecross_comp_dB[:,centerU], label=f"Ecross_comp u = 0 comp")
ax1.legend()
ax1.grid()
#ax1.set_ylim(-60,0)
ax1.set_xlabel("v")

ax2.plot(u[centerV,:], Eco_dB[centerV,:], label=f"Eco v = 0")
ax2.plot(u[centerV,:], Eco_comp_dB[centerV,:], label=f"Eco_comp v = 0 comp")
ax2.legend()
ax2.grid()
#ax2.set_ylim(-60,0)
ax2.set_xlabel("u")

ax3.plot(u[centerV,:], Ecross_dB[centerV,:], label=f"Ecross v = 0")
ax3.plot(u[centerV,:], Ecross_comp_dB[centerV,:], label=f"Ecross_comp v = 0 comp")
ax3.legend()
ax3.grid()
#ax3.set_ylim(-60,0)
ax3.set_xlabel("u")

Text(0.5,0,'u')

## $E_{RHCP}\,E_{LHCP}$

In [49]:
E_RHCP_comp = (E_theta_comp + 1j * E_phi_comp) / 2
E_LHCP_comp = (E_theta_comp - 1j * E_phi_comp) / 2

E_RHCP = (E_theta + 1j * E_phi) / 2
E_LHCP = (E_theta - 1j * E_phi) / 2

max_rhcp_lhcp      = np.amax((np.abs(E_LHCP),      np.abs(E_RHCP)     ))
max_rhcp_lhcp_comp = np.amax((np.abs(E_LHCP_comp), np.abs(E_RHCP_comp)))

E_LHCP      = E_LHCP      / max_rhcp_lhcp
E_RHCP      = E_RHCP      / max_rhcp_lhcp
E_LHCP_comp = E_LHCP_comp / max_rhcp_lhcp
E_RHCP_comp = E_RHCP_comp / max_rhcp_lhcp

In [50]:
dia.plot_E_LHCP_E_RHCP(u, v, E_LHCP, E_RHCP, nam_012, nam_011, vmax=0, vmin=-40)

In [51]:
dia.plot_E_LHCP_E_RHCP(u, v, E_LHCP_comp, E_RHCP_comp, nam_012, nam_011, vmax=0, vmin=-40, suffix=" comp")

In [52]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

ax0.plot(v[:,centerU], 20*np.log10(np.abs(E_LHCP[:,centerU])), label=f"LHCP u = 0")
ax0.plot(v[:,centerU], 20*np.log10(np.abs(E_LHCP_comp[:,centerU])), label=f"LHCP u = 0 comp")
ax0.legend()
ax0.grid()
ax0.set_ylim(-60,0)
ax0.set_xlabel("v")

ax1.plot(v[:,centerU], 20*np.log10(np.abs(E_RHCP[:,centerU])), label=f"RHCP u = 0")
ax1.plot(v[:,centerU], 20*np.log10(np.abs(E_RHCP_comp[:,centerU])), label=f"RHCP u = 0 comp")
ax1.legend()
ax1.grid()
ax1.set_ylim(-60,0)
ax1.set_xlabel("v")

ax2.plot(u[centerV,:], 20*np.log10(np.abs(E_LHCP[centerV,:])), label=f"LHCP v = 0")
ax2.plot(u[centerV,:], 20*np.log10(np.abs(E_LHCP_comp[centerV,:])), label=f"LHCP v = 0 comp")
ax2.legend()
ax2.grid()
ax2.set_ylim(-60,0)
ax2.set_xlabel("u")

ax3.plot(u[centerV,:], 20*np.log10(np.abs(E_RHCP[centerV,:])), label=f"RHCP v = 0")
ax3.plot(u[centerV,:], 20*np.log10(np.abs(E_RHCP_comp[centerV,:])), label=f"RHCP v = 0 comp")
ax3.legend()
ax3.grid()
ax3.set_ylim(-60,0)
ax3.set_xlabel("u")

Text(0.5,0,'u')

In [53]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

ax0.plot(phi[:,centerU]*180/pi, 20*np.log10(np.abs(E_LHCP[:,centerU])), label=f"LHCP theta = 0")
ax0.plot(phi[:,centerU]*180/pi, 20*np.log10(np.abs(E_LHCP_comp[:,centerU])), label=f"LHCP theta = 0 comp")
ax0.legend()
ax0.grid()
ax0.set_ylim(-60,0)
ax0.set_xlabel("phi")

ax1.plot(phi[:,centerU]*180/pi, 20*np.log10(np.abs(E_RHCP[:,centerU])), label=f"RHCP theta = 0")
ax1.plot(phi[:,centerU]*180/pi, 20*np.log10(np.abs(E_RHCP_comp[:,centerU])), label=f"RHCP theta = 0 comp")
ax1.legend()
ax1.grid()
ax1.set_ylim(-60,0)
ax1.set_xlabel("phi")

ax2.plot(theta[centerV,:]*180/pi, 20*np.log10(np.abs(E_LHCP[centerV,:])), label=f"LHCP phi = 0")
ax2.plot(theta[centerV,:]*180/pi, 20*np.log10(np.abs(E_LHCP_comp[centerV,:])), label=f"LHCP phi = 0 comp")
ax2.legend()
ax2.grid()
ax2.set_ylim(-60,0)
ax2.set_xlabel("theta")

ax3.plot(theta[centerV,:]*180/pi, 20*np.log10(np.abs(E_RHCP[centerV,:])), label=f"RHCP phi = 0")
ax3.plot(theta[centerV,:]*180/pi, 20*np.log10(np.abs(E_RHCP_comp[centerV,:])), label=f"RHCP phi = 0 comp")
ax3.legend()
ax3.grid()
ax3.set_ylim(-60,0)
ax3.set_xlabel("theta")

Text(0.5,0,'theta')

# Probe compensation II

## Compute probe pattern

In [68]:
E_V_x =  E_V_theta * np.cos(theta) * np.cos(phi) - E_V_phi * np.sin(phi)
E_V_y =  E_V_theta * np.cos(theta) * np.sin(phi) + E_V_phi * np.cos(phi)
E_V_z = -E_V_theta * np.sin(theta)

Ey_probe = E_V_theta * np.cos(theta) * np.sin(phi) + E_V_phi * np.cos(phi)

E_H_x =  E_H_theta * np.cos(theta) * np.cos(phi) - E_H_phi * np.sin(phi)
E_H_y =  E_H_theta * np.cos(theta) * np.sin(phi) + E_H_phi * np.cos(phi)
E_H_z = -E_H_theta * np.sin(theta)

Ex_probe = E_H_theta * np.cos(theta) * np.cos(phi) - E_H_phi * np.sin(phi)

## $E_x\,E_y$

$F_z$ is not considered.

In [ ]:
def getFz(kx, ky, kz, Fx, Fy):
    Fz = np.zeros(Fx.shape, dtype=complex)
    idx = np.where(kz!=0)
    Fz[idx] = - ( kx[idx] * Fx[idx] + ky[idx] * Fy[idx] ) / kz[idx]
    return Fz

In [ ]:
Fz = getFz(mesh_kx, mesh_ky, kz, Fx, Fy)

In [ ]:
E2x_comp = kz / k0 * Fx / Ex_probe
E2y_comp = kz / k0 * Fy / Ey_probe
E2z_comp = 0
E2x_comp_dB = 20 * np.log10( np.abs( E2x_comp ) )
E2y_comp_dB = 20 * np.log10( np.abs( E2y_comp ) )

Ex = np.cos(theta) * Fx
Ey = np.cos(theta) * Fy
Ez = np.cos(theta) * Fz
Ex_dB = 20 * np.log10( np.abs( E2x ) )
Ey_dB = 20 * np.log10( np.abs( E2y ) )

vmax = np.amax( ( np.abs(E2x_comp[validIdx]), np.abs(E2y_comp[validIdx]) ) )
vmin = np.amin( ( np.abs(E2x_comp[validIdx]), np.abs(E2y_comp[validIdx]) ) )
print(f"vmax = {vmax:.3f}, vmin = {vmin:.3f}")
vmax = 20 * np.log10( vmax )
vmin = 0
print(f"vmax = {vmax:.1f}, vmin = {vmin} [dB]")

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

ax0.plot(v[:,centerU], Ex_dB[:,centerU], label=f"E_x u = 0")
ax0.plot(v[:,centerU], E2x_comp_dB[:,centerU], label=f"E2x_comp_dB u = 0 comp")
ax0.legend()
ax0.grid()
#ax0.set_ylim(-60,0)
ax0.set_xlabel("v")

ax1.plot(v[:,centerU], Ey_dB[:,centerU], label=f"E_y u = 0")
ax1.plot(v[:,centerU], E2y_comp_dB[:,centerU], label=f"E2y_comp_dB u = 0 comp")
ax1.legend()
ax1.grid()
#ax1.set_ylim(-60,0)
ax1.set_xlabel("v")

ax2.plot(u[centerV,:], Ex_dB[centerV,:], label=f"Ex_dB v = 0")
ax2.plot(u[centerV,:], E2x_comp_dB[centerV,:], label=f"E2x_comp_dB v = 0 comp")
ax2.legend()
ax2.grid()
#ax2.set_ylim(-60,0)
ax2.set_xlabel("u")

ax3.plot(u[centerV,:], Ey_dB[centerV,:], label=f"Ey_dB v = 0")
ax3.plot(u[centerV,:], E2y_comp_dB[centerV,:], label=f"E2y_comp_dB v = 0 comp")
ax3.legend()
ax3.grid()
#ax3.set_ylim(-60,0)
ax3.set_xlabel("u")

## $E_{\theta}\,E_{\phi}$

In [ ]:
E2_r     =   Ex_comp * np.sin(theta) * np.cos(phi) + Ey_comp * np.sin(theta) * np.sin(phi) + Ez_comp * np.cos(theta)
E2_theta =   Ex_comp * np.cos(theta) * np.cos(phi) + Ey_comp * np.cos(theta) * np.sin(phi) - Ez_comp * np.sin(theta)
E2_phi   = - Ex_comp * np.sin(phi)                 + Ey_comp * np.cos(phi)

E2_theta_dB = 20 * np.log10( np.abs( E2_theta ) )
E2_phi_dB = 20 * np.log10( np.abs( E2_phi ) )

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

ax0.plot(v[:,centerU], E_theta_dB[:,centerU], label=f"E_theta u = 0")
ax0.plot(v[:,centerU], E2_theta_dB[:,centerU], label=f"E2_theta u = 0 comp")
ax0.legend()
ax0.grid()
#ax0.set_ylim(-60,0)
ax0.set_xlabel("v")

ax1.plot(v[:,centerU], E_phi_dB[:,centerU], label=f"E_phi u = 0")
ax1.plot(v[:,centerU], E2_phi_dB[:,centerU], label=f"E2_phi u = 0 comp")
ax1.legend()
ax1.grid()
#ax1.set_ylim(-60,0)
ax1.set_xlabel("v")

ax2.plot(u[centerV,:], E_theta_dB[centerV,:], label=f"E_theta v = 0")
ax2.plot(u[centerV,:], E2_theta_dB[centerV,:], label=f"E2_theta v = 0 comp")
ax2.legend()
ax2.grid()
#ax2.set_ylim(-60,0)
ax2.set_xlabel("u")

ax3.plot(u[centerV,:], E_phi_dB[centerV,:], label=f"E_phi v = 0")
ax3.plot(u[centerV,:], E2_phi_dB[centerV,:], label=f"E2_phi v = 0 comp")
ax3.legend()
ax3.grid()
#ax3.set_ylim(-60,0)
ax3.set_xlabel("u")

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2)

im0 = ax0.pcolor(u, v, 20*np.log10(np.abs(E2_theta)), vmax=vmax, vmin=vmin)
ax0.set_title("E2_theta")
ax0.set_xlabel("u")
ax0.set_ylabel("v")
dia.addColorBar(im0, ax0)

im1 = ax1.pcolor(u, v, 20*np.log10(np.abs(E2_phi)), vmax=vmax, vmin=vmin)
ax1.set_title("E2_phi")
ax1.set_xlabel("u")
ax1.set_ylabel("v")
dia.addColorBar(im1, ax1)

title = nam_011 +"\n" + nam_012
fig.suptitle(title)

## $E_{RHCP}\,E_{LHCP}$

In [ ]:
E2_RHCP = (E2_theta + 1j * E2_phi) / 2
E2_LHCP = (E2_theta - 1j * E2_phi) / 2

max_rhcp_lhcp = np.amax((np.abs(E2_LHCP), np.abs(E2_RHCP)))

E2_RHCP = E2_RHCP / max_rhcp_lhcp
E2_LHCP = E2_LHCP / max_rhcp_lhcp

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

ax0.plot(v[:,centerU], 20*np.log10(np.abs(E_LHCP[:,centerU])), label=f"LHCP u = 0")
ax0.plot(v[:,centerU], 20*np.log10(np.abs(E2_LHCP[:,centerU])), label=f"LHCP2 u = 0 comp")
ax0.legend()
ax0.grid()
#ax0.set_ylim(-60,0)
ax0.set_xlabel("v")

ax1.plot(v[:,centerU], 20*np.log10(np.abs(E_RHCP[:,centerU])), label=f"RHCP u = 0")
ax1.plot(v[:,centerU], 20*np.log10(np.abs(E2_RHCP[:,centerU])), label=f"RHCP2 u = 0 comp")
ax1.legend()
ax1.grid()
#ax1.set_ylim(-60,0)
ax1.set_xlabel("v")

ax2.plot(u[centerV,:], 20*np.log10(np.abs(E_LHCP[centerV,:])), label=f"LHCP v = 0")
ax2.plot(u[centerV,:], 20*np.log10(np.abs(E2_LHCP[centerV,:])), label=f"LHCP2 v = 0 comp")
ax2.legend()
ax2.grid()
#ax2.set_ylim(-60,0)
ax2.set_xlabel("u")

ax3.plot(u[centerV,:], 20*np.log10(np.abs(E_RHCP[centerV,:])), label=f"RHCP v = 0")
ax3.plot(u[centerV,:], 20*np.log10(np.abs(E2_RHCP[centerV,:])), label=f"RHCP2 v = 0 comp")
ax3.legend()
ax3.grid()
#ax3.set_ylim(-60,0)
ax3.set_xlabel("u")

# Probe compensation III

## Compute probe pattern

In [58]:
E_V_co    = E_V_theta * np.cos(phi) - E_V_phi * np.sin(phi)
E_V_cross = E_V_theta * np.sin(phi) + E_V_phi * np.cos(phi)

E_H_co    = E_H_theta * np.cos(phi) - E_H_phi * np.sin(phi)
E_H_cross = E_H_theta * np.sin(phi) + E_H_phi * np.cos(phi)

## $E_{co}\,E_{cross}$

In [69]:
E3_co       = np.cos(theta) * Fx / E_H_co
E3_cross    = np.cos(theta) * Fy / E_V_cross
E3_co_dB    = 20 * np.log10( np.abs( E3_co ) )
E3_cross_dB = 20 * np.log10( np.abs( E3_cross ) )

In [70]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

ax0.plot(v[:,centerU], E_co_dB[:,centerU], label=f"E_co u = 0")
ax0.plot(v[:,centerU], E3_co_dB[:,centerU], label=f"E3_cross u = 0 comp")
ax0.legend()
ax0.grid()
#ax0.set_ylim(-60,0)
ax0.set_xlabel("v")

ax1.plot(v[:,centerU], E_cross_dB[:,centerU], label=f"E_cross u = 0")
ax1.plot(v[:,centerU], E3_cross_dB[:,centerU], label=f"E3_cross u = 0 comp")
ax1.legend()
ax1.grid()
#ax1.set_ylim(-60,0)
ax1.set_xlabel("v")

ax2.plot(u[centerV,:], E_co_dB[centerV,:], label=f"Eco v = 0")
ax2.plot(u[centerV,:], E3_co_dB[centerV,:], label=f"E3_co v = 0 comp")
ax2.legend()
ax2.grid()
#ax2.set_ylim(-60,0)
ax2.set_xlabel("u")

ax3.plot(u[centerV,:], E_cross_dB[centerV,:], label=f"E_cross v = 0")
ax3.plot(u[centerV,:], E3_cross_dB[centerV,:], label=f"E3_cross_dB v = 0 comp")
ax3.legend()
ax3.grid()
#ax3.set_ylim(-60,0)
ax3.set_xlabel("u")

Text(0.5,0,'u')

## $E_{RHCP}\,E_{LHCP}$

In [71]:
E3_RHCP = (E3_co + 1j * E3_cross) / 2
E3_LHCP = (E3_co - 1j * E3_cross) / 2

max_rhcp_lhcp = np.amax((np.abs(E3_LHCP), np.abs(E3_RHCP)))

E3_RHCP = E3_RHCP / max_rhcp_lhcp
E3_LHCP = E3_LHCP / max_rhcp_lhcp

In [72]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

ax0.plot(v[:,centerU], 20*np.log10(np.abs(E_LHCP[:,centerU])), label=f"LHCP u = 0")
ax0.plot(v[:,centerU], 20*np.log10(np.abs(E3_LHCP[:,centerU])), label=f"LHCP3 u = 0 comp")
ax0.legend()
ax0.grid()
#ax0.set_ylim(-60,0)
ax0.set_xlabel("v")

ax1.plot(v[:,centerU], 20*np.log10(np.abs(E_RHCP[:,centerU])), label=f"RHCP u = 0")
ax1.plot(v[:,centerU], 20*np.log10(np.abs(E3_RHCP[:,centerU])), label=f"RHCP3 u = 0 comp")
ax1.legend()
ax1.grid()
#ax1.set_ylim(-60,0)
ax1.set_xlabel("v")

ax2.plot(u[centerV,:], 20*np.log10(np.abs(E_LHCP[centerV,:])), label=f"LHCP v = 0")
ax2.plot(u[centerV,:], 20*np.log10(np.abs(E3_LHCP[centerV,:])), label=f"LHCP3 v = 0 comp")
ax2.legend()
ax2.grid()
#ax2.set_ylim(-60,0)
ax2.set_xlabel("u")

ax3.plot(u[centerV,:], 20*np.log10(np.abs(E_RHCP[centerV,:])), label=f"RHCP v = 0")
ax3.plot(u[centerV,:], 20*np.log10(np.abs(E3_RHCP[centerV,:])), label=f"RHCP3 v = 0 comp")
ax3.legend()
ax3.grid()
#ax3.set_ylim(-60,0)
ax3.set_xlabel("u")

Text(0.5,0,'u')

## $E_{\theta}\,E_{\phi}$

In [73]:
E3_theta =  E3_co * np.cos(phi) + E3_cross * np.sin(phi)
E3_phi   = -E3_co * np.sin(phi) + E3_cross * np.cos(phi)

## $E_x\,E_y$

In [ ]:
E3_x = 
E3_y = 